# Tarea DB

## Ejercicio 1

In [25]:
import Bio.PDB.PDBParser

parser = Bio.PDB.PDBParser(QUIET=True)

entrada = str(input("Teclea la ruta hacia el archivo input"))

structure = parser.get_structure("1KCW", entrada)

model = structure[0]

achain = model['A']

cysresidues = []

# Se recorre cada residuo hasta encontrar CYS

for residue in achain:
    
    if residue.get_resname() == 'CYS':
        
        cysresidues.append(residue)


# El ciclo for recorre todos los residuos y posibles combinaciones

# Variable para guardar indice de c2

indiceAnterior = -1

for c1 in cysresidues:
    
    c1index = c1.get_id()[1]
    
    # Si el indice de c1 es el mismo que c2 en la corrida anterior se salta el ciclo
    
    # Es para evitar repeticiones
    
    if (c1index == indiceAnterior):
        
        continue
    
    for c2 in cysresidues:
    
        c2index = c2.get_id()[1]
        
        # Si ambos indices son iguales, se salta el ciclo
        
        if (c1index == c2index):
            
            continue
        
        # Se obtiene atomo SG para comparar distancias
        
        if (c1['SG'] - c2['SG']) < 8.0:
            
            print("possible di-sulfide bond:", end=" ")
            
            print("Cys",c1index,"-", end=" ")
            
            print("Cys",c2index, end=" ")
            
            print(round(c1['SG'] - c2['SG'],2))
            
            indiceAnterior = c2.get_id()[1]

Teclea la ruta hacia el archivo input../test/archivos_trabajo/1kcw.pdb
possible di-sulfide bond: Cys 155 - Cys 181 2.0
possible di-sulfide bond: Cys 257 - Cys 338 2.03
possible di-sulfide bond: Cys 515 - Cys 541 2.02
possible di-sulfide bond: Cys 618 - Cys 699 2.03
possible di-sulfide bond: Cys 855 - Cys 881 2.02


## Ejercicio 2

In [3]:
import Bio.PDB.PDBParser

import sys

from Bio.PDB.Polypeptide import is_aa

parser = Bio.PDB.PDBParser(QUIET=True)

entrada = str(input("Teclea ruta hacia archivo de entrada"))

structure = parser.get_structure("BS3", entrada)

aChain = structure[0]['A']

bChain = structure[0]['B']

lysAChain = []

lysBChain = []

# Se obtienen todas las lisinas en chain A y B

for residuo in aChain:
    
    if residuo.get_resname() == "LYS":
        
        lysAChain.append(residuo)
        
for residuo in bChain: 
    
    if residuo.get_resname() == "LYS":
        
        lysBChain.append(residuo)
        
lysBChain = [r['NZ'] for r in lysBChain if is_aa(r)]

neighbors = Bio.PDB.NeighborSearch(lysBChain)

for res1 in lysAChain:
    
    if is_aa(res1):
        
        r1Nz = res1['NZ']
        
        r1Ind = res1.get_id()[1]
        
        for r2Nz in neighbors.search(r1Nz.get_coord(), 12):
            
            res2 = r2Nz.get_parent()
            
            r2Ind = res2.get_id()[1]
    
            print("LYS",r1Ind,"en chain A", end=" ")
        
            print("y LYS",r2Ind,"en chain B", end=" ")
            
            print("se encuentran cerca:", round(r1Nz-r2Nz,2))

Teclea ruta hacia archivo de entrada../test/archivos_ejercicio/1ce0.pdb
LYS 12 en chain A y LYS 7 en chain B se encuentran cerca: 11.29


## Ejercicio 3

In [73]:
from Bio import Entrez

Entrez.email = "angeldc@lcg.unam.mx"

handle = Entrez.einfo(db = "protein")

record = Entrez.read(handle)

# Recorre cada field en el record

for field in record["DbInfo"]["FieldList"]:
    
    if field["Name"] == "ECNO":
        
        print(f"# Descripcion de ECNO :")
        
        print(field["Description"])
        
# Recorre cada link en el record
        
for link in record["DbInfo"]["LinkList"]:
    
    if link["Name"] == "protein_protein_small_genome":
        
        print("# Descripcion de protein_protein_small_genome :")
        
        print(link["Description"])

# Descripcion de ECNO :
EC number for enzyme or CAS registry number
# Descripcion de protein_protein_small_genome :
All proteins from this genome


## Ejercicio 4

In [71]:
from Bio import Entrez, SeqIO

newFile = open("../build/outputEjercicioDB.fasta", "w+")

Entrez.email = "angeldc@lcg.unam.mx"

# Funcion para obtener secuencia de cada gen

def obtenerProteina (listaDeBusqueda, archivo):
    
    for elemento in listaDeBusqueda:
    
        handle = Entrez.esearch(db= "nucleotide", term = elemento)

        result = Entrez.read(handle)

        handle = Entrez.efetch(db = "nucleotide", id = result["IdList"], rettype = "gb", retmode = "text")

        record = SeqIO.read(handle, "genbank")

        handle.close()
        
        print(">", end = "")

        print(record.id, record.description)

        print(record.seq)
        
        print("\n\n")
        
        escribirArchivo = ">" + str(record.id) + " " + str(record.description) + "\n" + str(record.seq) + "\n"
        
        archivo.write(escribirArchivo)
        
# Crea lista con cada consulta a realizar
        
entrada = str(input("Teclea cada busqueda con una diagonal entre cada busqueda\n"))

lista = entrada.split("/")

print("\nLo siguiente sera escrito en el archivo output\n")

obtenerProteina(lista, newFile)



Teclea cada busqueda con una diagonal entre cada busqueda
Notoryctes typhlops [Organism] AND PNOC [Gene Name]/ Notoryctes typhlops [Organism] AND RUNX2 [Gene Name]/Cypripedium guttatum [Organism] AND matK [Gene Name]/Hepatitis B virus [Organism] AND X [Gene Name] AND migrant 467 [Isolate]

Lo siguiente sera escrito en el archivo output

>JN633765.1 Notoryctes typhlops prepronociceptin (PNOC) gene, partial cds
TTCTGGGCTCCTTGCATCAAGGCCATGAATAAGGGGCCCTGGCAGCTCAGCTCAGCCAGCAAGGACAAGGCAATGACATCCTTCTATCAGCCACAGTTGATGGAGGCACAATCCCTAAGCCAGATGGAGGGCAAGAACCGAGTGTCTCGTGTCAGAAGTTTGTTCCATGCCCAGGAACAAGAGGCAGCAGAAGGCTTAGGGGAGTCAGGGGAAGAGATGCAGAAGAAATTACAGAAACGGTTTGGGGGCTTCACTGGGGCACGGAAATCAGCCCGGAAGTTGGCCAACCAGAAG



>KY938699.1 Notoryctes typhlops runt-related transcription factor 2 (RUNX2) gene, partial cds
ACCCGGCAGATGAGCGACGTGAGCCCGGTAGTGGCTGCCCAGCAGCAGCAGCAGCAGCAGCAGCAGCAACAGCAACAGCAGCAACAGCAGCAGCAGCAGCAGCAGCAGCAGGAGGCGGCGGCCGCAGCAGCGGCGGCGGCGGCGGCGGCGGCGGCGGCGGCAGCGGCAGCGGCAGCCGCGGTGCCGAGGCTGCGC